Installing Libraries & Packages - 

In [2]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax


## Twitter-roBERTa-base for Sentiment Analysis

Below I use the roBERTa-base model trained on ~58M tweets and finetuned for sentiment analysis with the TweetEval benchmark. This model is suitable for English.

In [4]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

### Testing the model -

In [6]:
sample = "This was incredibly bad"
def roberta_senanalyis(inp):
    encoded_text = tokenizer(inp, return_tensors='pt')
    output = model(**encoded_text)
    scores = softmax(output[0][0].detach().numpy())
    scores_dict = {
        'negative' : scores[0],
        'neutral' : scores[1],
        'positive' : scores[2]
    }
    return scores_dict

neg, neu, pos = roberta_senanalyis(sample)['negative'], roberta_senanalyis(sample)['neutral'], roberta_senanalyis(sample)['positive']
maxm = max(neg, neu, pos)
print(neg, neu, pos)

0.9769726 0.020274766 0.0027526724


It can be seen above that the model gives a negative sentiment for the sample input "This was incredibly bad".

### Testing Hugging Face's Sentiment Analysis Pipeline - 

The pipelines are a great and easy way to use models for inference. These pipelines are objects that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering. 

In [7]:
sentiment_analysis = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [8]:
sentiment_analysis("I kind of liked that song.") # Positive sentiment

[{'label': 'POSITIVE', 'score': 0.999756395816803}]

In [9]:
sentiment_analysis("beautiful mess") # Not so perfect (?)

[{'label': 'NEGATIVE', 'score': 0.826908528804779}]

## Grammatical Error Correction using Happytransformer from Hugging Face -

In [10]:
!pip install happytransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.9 MB/s eta 0:00:00


In [11]:
from happytransformer import HappyTextToText, TTSettings

In [12]:
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

args = TTSettings(num_beams=5, min_length=1)

In [13]:
result = happy_tt.generate_text("grammar: This sentences has has bads grammar", args=args)

print(result.text)

This sentence has bad grammar.


In [15]:
def compiler(ip):
  neg, neu, pos = roberta_senanalyis(ip)['negative'], roberta_senanalyis(ip)['neutral'], roberta_senanalyis(ip)['positive']
  maxm = max(neg, neu, pos)
  fixed = happy_tt.generate_text("grammar: {}".format(ip),args=args).text
  print("Corrected sentence: ", fixed)
  print("Mood: ", end='')
  if maxm == neg:
    print('😒')
  elif maxm == neu:
    print('😐')
  else:
    print('😀')


# Testing for mood & grammatical errors in input -

In [16]:
compiler('I mean we all sorts messed') # Negative

Corrected sentence:  I mean, we all sort of messed up.
Mood: 😒


In [18]:
compiler('What was Goin the clstroom was bad') # Negative

Corrected sentence:  What was going in the classroom was bad.
Mood: 😒


In [19]:
compiler('That was such a goood times') # Positive

Corrected sentence:  That was such a great time.
Mood: 😀


In [26]:
compiler('I will go check') # Neutral

Corrected sentence:  I will go check it out.
Mood: 😐
